<a href="https://colab.research.google.com/github/HarshitSingh032/Twitter_Sentiment_Analysis/blob/main/Twitter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [3]:
  #uploading Kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

**Importing Twitter Sentiment datasets**

The dataset being used is the **sentiment140 dataset**. It contains** 1,600,000 tweets** extracted using the Twitter Dataset CSV file. The tweets have been annotated **(0 = Negative, 4 = Positive)** and they can be used to detect sentiment.

It contains the following 6 fields:

sentiment: the polarity of the tweet (0 = negative, 4 = positive)
1. **IDs**: The id of the tweet (2087)
2. **date**: the date of the tweet (Sat May 16 23:58:44 UTC 2009)
3. **flag**: The query (lyx). If there is no query, then this value is NO_QUERY.
4. **user**: the user that tweeted (robotickilldozr)
5. **text**: the text of the tweet (Lyx is cool)e cool)
We require only the sentiment and text fields, so we discard the rest.

In [4]:
#API to fetch dataset from Kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
  0% 0.00/80.9M [00:00<?, ?B/s]
100% 80.9M/80.9M [00:00<00:00, 1.18GB/s]


In [5]:
 #extracting the compressed dataset

from zipfile import ZipFile
dataset = '/content/sentiment140.zip'
with ZipFile(dataset,'r') as zip:  #r stands for read
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


**Importing the dependecies**

In [6]:
import numpy as np
import pandas as pd
import re #re stands for regular expressiion
from nltk.corpus import stopwords #nltk is natural language tool and corpus is the module
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


**Preprocess Text**
Text Preprocessing is traditionally an important step for **Natural Language Processing (NLP)** tasks. It transforms text into a more digestible form so that machine learning algorithms can perform better.

The Preprocessing steps taken are:}

1. **Lower Casing**: Each text is converted to lowercase.
2. **Replacing URLs**: Links starting with "http" or "https" or "www" are replaced by "URL".
3. **Replacing Emojis**: Replace emojis by using a pre-defined dictionary containing emojis along with their meaning. (eg: ":)" to "EMOJIsmile")
4. **Replacing Usernames**: Replace @Usernames with word "USER". (eg: "@Kaggle" to "USER")
5. **Removing Non-Alphabets**: Replacing characters except Digits and Alphabets with a space.
6. **Removing Consecutive letters**: 3 or more consecutive letters are replaced by 2 letters. (eg: "Heyyyy" to "Heyy")
7. **Removing Short Words**: Words with length less than 2 are removed.
8. **Removing Stopwords**: Stopwords are the English words which does not add much meaning to a sentence. They can safely be ignored without sacrificing the meaning of the sentence. (eg: "the", "he", "have")
9. **Lemmatizing**: Lemmatization is the process of converting a word to its base form. (e.g: “Great” to “Good”)

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [9]:
#loading data to pandas dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding = 'ISO-8859-1')

In [10]:
#checking the number of rows and column
twitter_data.shape

(1599999, 6)

In [11]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [12]:
 #since pd.read hasent read the coloumn names so now u are correcting it
 #naming the coloumns and reading tha dataset agin\

coloumn_names = ['target','ids','date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=coloumn_names,encoding = 'ISO-8859-1')

In [13]:
twitter_data.shape

(1600000, 6)

In [14]:
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [15]:
#now checking the missing values
twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [16]:
#now checking the distribution of target column basically checking the positive and negative tweets
twitter_data['target'].value_counts()  #since it has equal distribution so we dont have to do extra work

,count
target,
0,800000
4,800000


In [17]:
#now converting the label 4 to 1
#inplace is mentioned so that cgange should be in the original dataframe
twitter_data.replace({'target':{4:1}},inplace=True)
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


0--->negative tweet
1---> positive tweet

*Stemming*   It reduces a word to its root word

In [18]:
port_stem = PorterStemmer()


In [19]:
#def is define it defines a function
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)#removing all the content which are not allphabet
  stemmed_content = stemmed_content.lower()  #converting upper case to lower case
  stemmed_content = stemmed_content.split()#spliting to a list to check the stopwords
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]#if the word is a stopword then do not stem it this explain this
  stemmed_content = ' '.join(stemmed_content) #in this we join all the words of that tweet
  return stemmed_content



In [20]:
#applying the function to the data
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [21]:
twitter_data.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [22]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [23]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [24]:
#seperating the data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [25]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [26]:
print(Y)

[0 0 0 ... 1 1 1]


**Splitting the data to traing data and test data**
  
    A train/test split is a crucial technique in machine learning where a dataset is divided into two subsets: a training set and a testing set. The training set is used to train the model, while the testing set is used to evaluate its performance on unseen data. This helps assess how well the model generalizes and avoids overfitting or underfitting.

In [27]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)#using train split test data test size means how many data are used for test data  stratify means equal proportion of data in 0 and 1

In [28]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [29]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [30]:
print(X_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


**TF-IDF Vectoriser**

TF-IDF indicates what the importance of the word is in order to understand the document or dataset. Let us understand with an example. Suppose you have a dataset where students write an essay on the topic, My House. In this dataset, the word a appears many times; it’s a high frequency word compared to other words in the dataset. The dataset contains other words like home, house, rooms and so on that appear less often, so their frequency are lower and they carry more information compared to the word. This is the intuition behind TF-IDF.

 ---->Transforming the X_train and X_test dataset into matrix of TF-IDF Features by using the TF-IDF Vectoriser. This datasets will be used to train the model and test against it.

In [31]:
#converting the textual data to numerical data

vectorizer = TfidfVectorizer() #it assigns weightage toh words according to the formula of Tfidf
vectorizer.fit(X)

X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [32]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9453092 stored elements and shape (1280000, 532699)>
  Coords	Values
  (0, 126118)	0.37484032460318173
  (0, 213554)	0.5289928671316323
  (0, 271144)	0.42037613603065954
  (0, 409438)	0.35861719248416485
  (0, 504044)	0.2725664180116586
  (0, 511398)	0.4472655127083222
  (1, 185304)	1.0
  (2, 90030)	0.31341180319788786
  (2, 126118)	0.45825085123736603
  (2, 143635)	0.18906085394224742
  (2, 149331)	0.29041017783193285
  (2, 152669)	0.20261379420870737
  (2, 173898)	0.1880703874098935
  (2, 205312)	0.16175394132247145
  (2, 307827)	0.2410455646561603
  (2, 333015)	0.16780678648815192
  (2, 468994)	0.32099337313155857
  (2, 470034)	0.18720443014780438
  (2, 472107)	0.15171423197968206
  (2, 500375)	0.3297483694639193
  (2, 511810)	0.33617620658621283
  (3, 65151)	0.5205345321237522
  (3, 175126)	0.27847482359309217
  (3, 183099)	0.44549570924150195
  (3, 198842)	0.3743117172596608
  :	:
  (1279996, 336028)	0.17877262371579808

Training the machine learning model

**Logistic Regression** - Logistic regression is a supervised machine learning algorithm primarily used for binary classification problems. It predicts the probability of an instance belonging to a specific class (e.g., spam or not spam, disease or no disease).

In [33]:
model = LogisticRegression(max_iter=1000)

In [34]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

**Model Evaluation  by accuracy score**

In [35]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train,X_train_prediction)

In [36]:
print('Accuracy score of training data : ',training_data_accuracy)

Accuracy score of training data :  0.8047734375


In [37]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test,X_test_prediction)

In [38]:
print('Accuracy score of test data : ',test_data_accuracy)

Accuracy score of test data :  0.777475


**Model** **accuracy** = 77.7 %

**Saving the model**

In [39]:
import pickle

In [40]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb')) #wb write

**Using the saved model for future predictions**

In [41]:
#loading the saved model
loaded_model = pickle.load(open('trained_model.sav','rb'))

In [43]:
X_new = X_test[200]
print(Y_test[200])
prediction = model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print('The tweet is negative')
else:
  print('The tweet is positive')

1
[1]
The tweet is positive
